In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from visualize import visualize_rooms
from preprocessing import preprocess_df, merge_data
from sklearn.ensemble import RandomForestRegressor
from model import train_model
from autots import AutoTS
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from autots.evaluator.benchmark import Benchmark
bench = Benchmark()
bench.run(n_jobs="auto", times=3)
bench.results

In [ ]:
# load dataset and preprocess
df = pd.read_csv("data/train/data.csv", sep=";")
# merge dataset with weather data
df = merge_data(df)
# discretize, impute etc.
df = preprocess_df(df)

In [ ]:
# AutoRegressor

In [ ]:
# export
import torch
dfp.to_csv("./data/train/preprocessed_train.csv")
print(torch.cuda.is_available())

In [2]:
# import
df = pd.read_csv("./data/train/preprocessed_train.csv")

Iteration 1, loss = 6.12824612
Iteration 2, loss = 0.29221966
Iteration 3, loss = 0.22378249
Iteration 4, loss = 0.19912970
Iteration 5, loss = 0.18698228
Iteration 6, loss = 0.17930021
Iteration 7, loss = 0.17403015
Iteration 8, loss = 0.17046356
Iteration 9, loss = 0.16748388
Iteration 10, loss = 0.16504452
Iteration 11, loss = 0.16319854
Iteration 12, loss = 0.16072004
Iteration 13, loss = 0.15939499
Iteration 14, loss = 0.15775809
Iteration 15, loss = 0.15660681
Iteration 16, loss = 0.15510819
Iteration 17, loss = 0.15401781
Iteration 18, loss = 0.15316896
Iteration 19, loss = 0.15219019
Iteration 20, loss = 0.15147132
Iteration 21, loss = 0.15073778
Iteration 22, loss = 0.14995406
Iteration 23, loss = 0.14918801
Iteration 24, loss = 0.14902369
Iteration 25, loss = 0.14813625
Iteration 26, loss = 0.14774502
Iteration 27, loss = 0.14762732
Iteration 28, loss = 0.14694145
Iteration 29, loss = 0.14662518
Iteration 30, loss = 0.14629679
Iteration 31, loss = 0.14596139
Iteration 32, los

In [3]:
df = df.rename(columns={'Unnamed: 0': 'timestamp'})
#df.set_index('timestamp', inplace=True)

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [5]:
g = df.groupby("Room")
split = [g.get_group(x) for x in g.groups]
dfone = split[0].reset_index(drop=True)
dfone["time_idx"] = dfone.index

In [ ]:
print(dfone.columns)

In [ ]:
print(dfone["RoomTemperature"])

In [6]:
max_prediction_length = 4*24*7
max_encoder_length = 5*4*24*7
training_cutoff = dfone["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    dfone[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="RoomTemperature",
    group_ids=["Room"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    #static_reals=["Room"],
    time_varying_known_categoricals=[],
    time_varying_unknown_reals=["RoomTemperature"],
    #time_varying_known_reals=[
    #"time_idx",  'AirqualityPerc', 'HeaterPerc',
    #    'CoolerPerc', 'TempSupplyAir', 'RelativeHumiditySupplyAir',
    #    'HeatingPower', 'CoolingPower', 'AirTemperature', 'WindDirection',
    #    'BrightnessNorth', 'BrightnessEast', 'BrightnessSouth',
    #    'BrightnessWest', 'dwpt', 'rhum',
    #    'prcp', 'wspd', 'tsun'],
    # time_varying_unknown_categoricals=[],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, dfone, predict=True, stop_randomization=True)
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=10)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=10)

In [ ]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [7]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    hidden_size=16,
    lstm_layers=2,
    learning_rate=0.06760829753919811,
    dropout=0.1,
    loss=RMSE(),
    attention_head_size=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 21.5k


In [75]:
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

0.532465699366784


In [8]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=50,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    #fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | RMSE                            | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 80    
4  | static_variable_selection          | VariableSelectionNetwork        | 1.7 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.2 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 528   
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 1.1 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [4]:
# train your model then:
# set n=1 if you only want your best model
model.export_template("my_models.csv", models='best', n=15, max_per_model_class=3)

# later on a new session
# you can set `max_generations=0` in model, and then it will only attempt the imported models
# model = model.import_template("my_models.csv", method='only')